In [1]:
using Pkg
Pkg.activate("..")
using ElementalFunctionsTests

  Activating project at `~/Documents/Academia/taylor/ElementalFunctionsTests`


In [2]:
using ProgressMeter
using JSON
using Printf

In [3]:
JSON.lower(e::Error32) =Dict(:x=>@sprintf("%a", e.x), :err=>e.err)

In [4]:
const r = range(-2, 2, length=41)
errors = []

Any[]

In [5]:
for f in mpfrfunctions
    x = nextfloat(-Inf32)
    maxerror = Error32(0.0, 0.0)
    h = buckets(r)
    
    f_c = eval(:(@cfunction($f, Cfloat, (Cfloat,))))
    f_mpfr = mpfrfun(string(f))
    dom = domain(Symbol(f))
    prog = Progress(4278190078, showspeed=true, dt=2.0, desc=string(f))
    sum=0    # To store sum of stream 
    sumsq=0  # To store sum of square of stream 
    n=0   

    
    while(!isinf(x))
        next!(prog)
        if x < dom[1]
            x = nextfloat(x)
            continue
        elseif x > dom[2]
            break;
        end
        
        err = ulperror32(f_c, f_mpfr, x)
        add2bucket!(h, r, err)
        abserr = abs(err)
        n+=1
        sum+=abserr
        sumsq+=(abserr*abserr) 
        
        if abs(err)>abs(maxerror.err)
          maxerror = Error32(x, err)
        end
        x = nextfloat(x)
    end

    mean = sum/n 
    var = (sumsq/n) - (mean*mean) 
    
    finish!(prog)
      
    push!(errors, TestsResults(Symbol(f),r,h, maxerror, 0, mean, var))
    open("Float32Errors.json","w") do f
      write(f, JSON.json(errors))
    end
end

cos 100%|██████████████████████████████████| Time: 4:14:24 ( 3.57 μs/it)m
sin 100%|██████████████████████████████████| Time: 4:25:07 ( 3.72 μs/it)
tan 100%|██████████████████████████████████| Time: 4:44:01 ( 3.98 μs/it)
cospi 100%|████████████████████████████████| Time: 3:42:57 ( 3.13 μs/it)
sinpi 100%|████████████████████████████████| Time: 4:01:46 ( 3.39 μs/it)
acos 100%|█████████████████████████████████| Time: 2:37:32 ( 2.21 μs/it)
asin 100%|█████████████████████████████████| Time: 3:30:26 ( 2.95 μs/it)
atan 100%|█████████████████████████████████| Time: 4:32:14 ( 3.82 μs/it)
csc 100%|██████████████████████████████████| Time: 4:47:30 ( 4.03 μs/it)
sec 100%|██████████████████████████████████| Time: 4:15:41 ( 3.59 μs/it)
cot 100%|██████████████████████████████████| Time: 4:52:30 ( 4.10 μs/it)
cosh 100%|█████████████████████████████████| Time: 3:49:05 ( 3.21 μs/it)
sinh 100%|█████████████████████████████████| Time: 3:51:58 ( 3.25 μs/it)
tanh 100%|█████████████████████████████████| Time:

In [5]:
for f in [acosh, expm1]
    x = nextfloat(-Inf32)
    maxerror = Error32(0.0, 0.0)
    h = buckets(r)
    
    f_c = eval(:(@cfunction($f, Cfloat, (Cfloat,))))
    f_mpfr = mpfrfun(string(f))
    dom = domain(Symbol(f))
    prog = Progress(4278190078, showspeed=true, dt=2.0, desc=string(f))
    sum=0    # To store sum of stream 
    sumsq=0  # To store sum of square of stream 
    n=0   

    
    while(!isinf(x))
        next!(prog)
        if x < dom[1]
            x = nextfloat(x)
            continue
        elseif x > dom[2]
            break;
        end
        
        err = ulperror32(f_c, f_mpfr, x)
        add2bucket!(h, r, err)
        abserr = abs(err)
        n+=1
        sum+=abserr
        sumsq+=(abserr*abserr) 
        
        if abs(err)>abs(maxerror.err)
          maxerror = Error32(x, err)
        end
        x = nextfloat(x)
    end

    mean = sum/n 
    var = (sumsq/n) - (mean*mean) 
    
    finish!(prog)
      
    println(TestsResults(Symbol(f),r,h, maxerror, 0, mean, var))
end

acosh 100%|████████████████████████████████| Time: 2:07:05 ( 1.78 μs/it)


TestsResults(:acosh, -2.0:0.1:2.0, [7.0, 24.0, 62.0, 156.0, 428.0, 1084.0, 2061.0, 4430.0, 8972.0, 17762.0, 41593.0, 104116.0, 7.566408e6, 1.49999e7, 1.7832478e7, 3.6090028e7, 4.2502043e7, 6.9266262e7, 7.9047053e7, 1.07315403e8, 1.07319865e8, 1.07259229e8, 9.9801672e7, 9.2372951e7, 8.9541074e7, 7.1286965e7, 6.4864688e7, 3.8104983e7, 2.8313707e7, 41611.0, 17740.0, 9012.0, 4263.0, 1998.0, 1085.0, 463.0, 164.0, 64.0, 16.0, 3.0], Error32(1.0077478f0, 1.9944941297023218), 0, 0.3190273348924196, 0.05059992864930919)


expm1 100%|████████████████████████████████| Time: 3:38:19 ( 3.06 μs/it)


TestsResults(:expm1, -2.0:0.1:2.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 522.0, 306512.0, 5.0249318e7, 5.0067203e7, 4.9697323e7, 4.9386405e7, 4.9147767e7, 1.299106545e9, 6.4223502e7, 5.6960872e7, 5.3658204e7, 5.24033e7, 256006.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Error32(0.34657723f0, -0.6083148857518701), 0, 0.03960886248876376, 0.011352621212585401)
